# Attrs
> attrs is the Python package that will bring back the joy of writing classes by relieving you from the drudgery of implementing object protocols (aka dunder methods).
> 
> Its main goal is to help you to write concise and correct software without slowing down your code.

**REF**: https://www.attrs.org/en/stable/index.html

## Last Updated

2023-11-18

In [3]:
from attrs import define, field

In [2]:
# Differentiation of Objects!


@define
class Empty:
    pass


print(Empty())
print(Empty() == Empty())
print(Empty() is Empty())

Empty()
True
False


In [6]:
# Basic Example


@define
class Coordinates:
    x: int
    y: int


c1 = Coordinates(1, 2)
c2 = Coordinates(x=-2, y=0)

print(c1)
print(c2)
print(c1 == c2)

Coordinates(x=1, y=2)
Coordinates(x=-2, y=0)
False


In [51]:
# Private attributes!


@define
class C:
    _x: int


c1 = C(x=1)  # !!
print(c1)

try:
    c2 = C(_x=1)  # This breaks!
except TypeError as e:
    print(e)

C(_x=1)
C.__init__() got an unexpected keyword argument '_x'


In [52]:
# Don't allow the user to init a var!


@define
class C:
    _x: int = field(init=False, default=42)


print(C())

try:
    print(C(23))
except TypeError as e:
    print(e)

C(_x=42)
C.__init__() takes 1 positional argument but 2 were given


In [12]:
# Okay, but I like exposing my privates.


@define
class C:
    _x: int = field(alias="_x")


C(_x=1)  # Works now!

C(_x=1)

In [14]:
# Here's another way to do attrs with a class you can't directly access (eg, Django).


class SomethingFromSomeoneElse:
    def __init__(self, x):
        self.x = x


SomethingFromSomeoneElse = define(these={"x": field()}, init=False)(
    SomethingFromSomeoneElse
)

SomethingFromSomeoneElse(1)

SomethingFromSomeoneElse(x=1)

In [53]:
# Keyword-only Attributes


@define
class A:
    a: int = field(kw_only=True)


try:
    A(1)
except TypeError as e:
    print(e)

A(a=1)

A.__init__() takes 1 positional argument but 2 were given


A(a=1)

In [50]:
# But I'm so lazy, and I want all the things to be kw_only.


@define(kw_only=True)
class A:
    a: int
    b: int


try:
    A(1, 2)
except TypeError as e:
    print(e)

A(a=1, b=2)

A.__init__() takes 1 positional argument but 3 were given


A(a=1, b=2)

In [21]:
# I want a dict from my class.

from attrs import asdict

asdict(Coordinates(x=1, y=2))

{'x': 1, 'y': 2}

In [28]:
# But my field is sensitive.  What can I do for that...?


@define
class User:
    email: str
    password: str


@define
class UserList:
    users: list[User]


users = [User("jane@doe.invalid", "s33kred"), User("joe@doe.invalid", "p4ssw0rd")]
user_list = UserList(users=users)

print(user_list)
print(asdict(user_list))
print(
    asdict(user_list, filter=lambda attr, value: attr.name != "password")
)  # no password field.

UserList(users=[User(email='jane@doe.invalid', password='s33kred'), User(email='joe@doe.invalid', password='p4ssw0rd')])
{'users': [{'email': 'jane@doe.invalid', 'password': 's33kred'}, {'email': 'joe@doe.invalid', 'password': 'p4ssw0rd'}]}
{'users': [{'email': 'jane@doe.invalid'}, {'email': 'joe@doe.invalid'}]}


In [40]:
from attrs import asdict, filters, fields


@define
class User:
    login: str
    password: str
    email: str
    id: int


user_jane = User("jane", "s33kred", "jane@example.com", 42)
asdict(
    user_jane, filter=filters.exclude(fields(User).password, int)
)  # Excludes password, and any int fields.

{'login': 'jane', 'email': 'jane@example.com'}

In [41]:
# Similarly...

from attrs import astuple

astuple(user_jane)

('jane', 's33kred', 'jane@example.com', 42)

In [44]:
# Validation method 1.


@define
class C:
    x: int = field()

    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("Oh no dudeee.")


try:
    C(100)
except ValueError as e:
    print(e)

print(C(41))

It worked!
C(x=41)


In [59]:
# Validation Method 2.

from attrs import validators


def x_not_greater_than_42_or_y(instance, attribute, value):
    if (value > 42) or (value > instance.y):
        raise ValueError("'x' has to be leq 42 and 'y'.")


@define
class C:
    x: int = field(validator=[validators.instance_of(int), x_not_greater_than_42_or_y])
    y: int


print(C(x=4, y=5))

try:
    C(x=43, y=55)
except ValueError as e:
    print(e)

try:
    C(x=41, y=40)
except ValueError as e:
    print(e)

try:
    C(x="hi", y=20)
except TypeError as e:
    print(e)

c_example = C(x=1, y=2)
try:
    c_example.x = 3  # Validation on setting.
except ValueError as e:
    print(e)

C(x=4, y=5)
'x' has to be leq 42 and 'y'.
'x' has to be leq 42 and 'y'.
("'x' must be <class 'int'> (got 'hi' that is a <class 'str'>).", Attribute(name='x', default=NOTHING, validator=_AndValidator(_validators=(<instance_of validator for type <class 'int'>>, <function x_not_greater_than_42_or_y at 0x7fa167db6cb0>)), repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'int'>, converter=None, kw_only=False, inherited=False, on_setattr=None, alias='x'), <class 'int'>, 'hi')
'x' has to be leq 42 and 'y'.
